In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import ipywidgets as widgets
import cartopy
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import matplotlib.pyplot as plt
from utils import sampling
from scipy.stats import theilslopes
from ismn.interface import ISMN_Interface
from pytesmo.df_metrics import ubrmsd
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import utils
from matplotlib.gridspec import GridSpec
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import datetime

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Data sets for this lecture

## Summary

In this lecture we will compute the following soil moisture anomaly based drought indices using monthly soil moisture data from C3S SM (v202012, downloaded from DOI: 10.24381/cds.d7782f18, and v202212):

* Soil Moisture anomalies
* Anomaly Quantiles
* Z-Scores

We will also use pre-processed Soil Moisture Anomalies Stantardised Index (SMASI) data.

In addition we will use ECMWF ERA-5 monthly precipitation data to compute:

* Standardised Precipitation Index (SPI)

Finally we will use soil moisture and Vegetation Optical Depth (VOD) time series to assess temporal lags between them.

First we load our soil moisture stack. In this case use the same stack as the one from the previous lecture on satellite soil moisture retrieval. The data is produced as part of the Copernicus Climate Change Service (C3S), which combines retrievals from multiple passive sensors. The used subset contains monthly mean values for Europe from 1978 to September 2022.
More information on this data can be found in the [C3S Data Store](https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-soil-moisture?tab=overview).

In [2]:
c3s_europe = xr.open_dataset('./LTC_DATA/STACK_C3S-SOILMOISTURE_EUROPE_v202212_PASSIVE_MONTHLY.nc')
display(c3s_europe)

<xarray.Dataset>
Dimensions:  (lat: 160, lon: 240, time: 530)
Coordinates:
  * lat      (lat) float32 69.88 69.62 69.38 69.12 ... 30.88 30.62 30.38 30.12
  * lon      (lon) float32 -9.875 -9.625 -9.375 -9.125 ... 49.38 49.62 49.88
  * time     (time) datetime64[ns] 1978-11-01 1978-12-01 ... 2022-12-01
Data variables:
    sm       (time, lat, lon) float32 ...
    nobs     (time, lat, lon) float32 ...
    sm_anom  (time, lat, lon) float32 ...
Attributes: (12/44)
    title:                        C3S Surface Surface Soil Moisture merged PA...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       LPRMv7/SMMR/Nimbus 7 L3 Surface Soil Moistu...
    platform:                     Nimbus 7, DMSP, TRMM, AQUA, Coriolis, GCOM-...
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    id:                           C3S-SOILMOISTURE-L3S-SSMV-PASSIVE-MONTHLY-1...
    history:                      2023-06-02T15:23:23.165973 mean calculated
    date_created:                 2023-06-02T15:23:23Z
    time_coverage_duration:       P1M
    time_coverage_start:          1978-10-31T12:00:00Z
    time_coverage_end:            1978-11-30T12:00:00Z

In addition we load precipitation data from ECMWF's ERA5 reanalysis (called `total precipitation` or `tp`). We have downloaded monthly averages for Europe in a similar time range as the soil moisture data. We sum up the (average) rainfall amount for each month. More background information on how to correctly sum-up precipitation data is available in the [support portal](https://confluence.ecmwf.int/pages/viewpage.action?pageId=197702790). 
ERA5 monthly means are available through the [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=overview).

In [3]:
era5_europe = xr.open_dataset('./LTC_DATA/era5_tp.nc').sel(time=slice('1978-01-01',None))

days_in_month = pd.DatetimeIndex(era5_europe.time).days_in_month.values
era5_europe['tp'].values = era5_europe['tp'].values * 1000 * days_in_month.reshape(len(days_in_month), 1, 1)
era5_europe = era5_europe.rename(dict(longitude="lon", latitude="lat"))
display(era5_europe)

<xarray.Dataset>
Dimensions:  (lon: 241, lat: 161, time: 548)
Coordinates:
  * lon      (lon) float32 -10.0 -9.75 -9.5 -9.25 -9.0 ... 49.25 49.5 49.75 50.0
  * lat      (lat) float32 70.0 69.75 69.5 69.25 69.0 ... 30.75 30.5 30.25 30.0
  * time     (time) datetime64[ns] 1978-01-01 1978-02-01 ... 2023-08-01
Data variables:
    tp       (time, lat, lon) float64 64.75 65.48 65.9 67.05 ... 0.0 0.0 0.0 0.0

# Loading data for the following examples
Droughts are extreme events in time. We therefore use time series as inputs to compute the indices. The following example allows you to extract the soil moisture and precipitation data for a chosen point in Europe. The left plot shows the location of the extracted time series.
We extract a time series for both data sets from our stacks and combine them in a pandas DataFrame for further processing. The figure on the right shows the extracted values for the chosen location and time period.

**Try:**

* **Changing the coordinates and see what a time series for a different location than the default one looks like. Find a suitable time series for the rest of the lecture or keep the default one in the area West of Wroclaw with a mixture of cultivated land and forest patches over a gentle topography.**

You can always reset the interactive controls by re-running the cell.

<font color='red'>**Note: The data you finally choose here will be used in the rest of the notebook**</font>

In [4]:
extracted_ts = dict()

slider=widgets.IntRangeSlider(min=1979, max=2022, value=[1991, 2022], step=1, description='Time series range (year from, year to)', 
                              continuous_update=True, style={'description_width': 'initial'}, layout=widgets.Layout(width='30%'))
@widgets.interact(period=slider, Longitude="16.5", Latitude="51.0")
def extract_ts(period, Longitude, Latitude):
    lon, lat = float(Longitude), float(Latitude)
    
    # Extract a pandas time series at location
    ts_sm = c3s_europe.sel(lon=lon, lat=lat, method='nearest') \
                   .to_pandas().rename(columns={'sm': 'soil_moisture'}) \
                   .loc[f"{period[0]}-01-01":f"{period[1]}-12-31", ['soil_moisture']]
    
    ts_tp = era5_europe.sel(lon=lon, lat=lat, method='nearest') \
               .to_pandas().rename(columns={'tp': 'precipitation'}) \
               .loc[f"{period[0]}-01-01":f"{period[1]}-12-31", ['precipitation']]
    
    ts = pd.concat([ts_sm, ts_tp], axis=1).resample('MS').mean()

    # Set up figures
    fig = plt.figure(figsize=(15,4), constrained_layout=True)
    gs = fig.add_gridspec(1, 3)
    map_ax = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree())
    ts_ax = fig.add_subplot(gs[0, 1:])
    bar_ax = ts_ax.twinx()

    # Plot overview map
    map_ax.set_extent([-14, 46, 29, 71])
    map_ax.add_feature(cartopy.feature.LAND, zorder=0)
    map_ax.add_feature(cartopy.feature.BORDERS)
    map_ax.coastlines()
    map_ax.plot([lon], [lat], 'ro', transform=ccrs.PlateCarree())
    map_ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
    
    # Plot time series
    ts['soil_moisture'].plot(style='-o', ax=ts_ax, label='soil moisture')
    ts['precipitation'].plot.area(ax=bar_ax, alpha=0.25, label='precipitation', linewidth=0)
    
    ts_ax.set_xlabel('Time [Year]')
    ts_ax.set_ylabel(f'Soil moisture $[m³/m³]$')
    bar_ax.set_ylabel(f'Precipitation $[mm]$')
    ts_ax.set_title('Satellite Soil Moisture and ERA5 Precipitation')
    
    ts_ax.legend(loc='upper left')
    bar_ax.legend(loc='upper right')
    
    # Make data available for the next example
    global extracted_ts
    extracted_ts = dict(data=ts.copy(), lon=lon, lat=lat)

interactive(children=(IntRangeSlider(value=(1991, 2022), description='Time series range (year from, year to)',…

# Compute anomalies for the chosen location
Here we use the time series that was extracted in the previous example.

We now compute the average soil moisture conditions---**climatology** ($\overline{SM}$)---for each month using data from all years within the selected reference period. 
The deviation of the absolute values ($SM$) from this climatological reference are the soil moisture **anomalies**. Values close to 0 therefore indicate normal conditions, while negative anomalies indicate drier than usual conditions (droughts). In case of strong precipitation events, positive anomalies are found.

$\Large SMA = SM_{k,i} - \overline{SM_i}$

**Try:**
    
* **Changing the baseline / reference period to see if / how it affects the climatology computation.**
* **Changing the amount of smoothing applied to the climatology.**

*Note: You can always reset the interactive controls by re-running the cell.*

In [78]:
anomaly_ts = None

@widgets.interact(baseline=widgets.IntRangeSlider(min=1991, max=2022, value=[1991, 2022], step=1, style={'description_width': 'initial'}, 
                                                  description='Reference period (year from, year to):', layout=widgets.Layout(width='30%')), 
                  smoothing=widgets.Dropdown(options=['strong', 'medium', 'none'], value='medium', style={'description_width': 'initial'}, description='Smoothing of climatology'))
def plot_components(baseline, smoothing):
    """
    Compute climatology and anomalies for the loaded soil moisture time series
    """
    loc = f"Lon: {extracted_ts['lon']}°, Lat: {extracted_ts['lat']}°" 
    
    fig, axs = plt.subplots(3, 1, figsize=(10, 7))
    ts = extracted_ts['data'].copy().loc['1991-01-01':,:]
    
    clim_data = ts['soil_moisture'].loc[f'{baseline[0]}-01-01':f'{baseline[1]}-12-31']
    clim_std = pd.Series(clim_data.groupby(clim_data.index.month).std(), name='climatology_std')      
    clim_mean = pd.Series(clim_data.groupby(clim_data.index.month).mean(), name='climatology')
    
    if smoothing.lower() == 'medium':
        window = 3
    elif smoothing.lower() == 'strong':
        window = 5
    else:
        window = 0
    
    if window != 0:
        clim_mean = clim_mean.rolling(window, min_periods=1).mean()
        
    ts = ts.join(on=ts.index.month, other=clim_mean)
    ts = ts.join(on=ts.index.month, other=clim_std)
    
    ts['anomaly'] = ts['soil_moisture'] - ts['climatology']
    
    ts['soil_moisture'].plot(ax=axs[0], title=f'Soil Moisture (absolute) at {loc}', ylabel=f'SM $[m³/m³]$', xlabel='Time [Year]')
    for i, g in clim_data.groupby(clim_data.index.year):
        axs[1].plot(range(1,13), g.values, alpha=0.2)
        
    clim_mean.plot(ax=axs[1], color='blue', title=f'Climatology at {loc}', ylabel='SM $[m³/m³]$', xlabel='Time [Month]', label='mean')
    clim_std.plot(ax=axs[1], label='$\sigma$')
    axs[1].legend()
    
    global anomaly_ts
    anomaly_ts = ts.copy()
    
    axs[2].axhline(0, color='k')
    axs[2].fill_between(ts['anomaly'].index,ts['anomaly'].values,where=ts['anomaly'].values>=0, color='blue')
    axs[2].fill_between(ts['anomaly'].index,ts['anomaly'].values,where=ts['anomaly'].values<0, color='red')
    axs[2].set_ylabel('SM $[m³/m³]$')
    axs[2].set_xlabel('Time [Year]')
    axs[2].set_title(f"SM Anomalies at {loc}")
    plt.tight_layout()


interactive(children=(IntRangeSlider(value=(1991, 2022), description='Reference period (year from, year to):',…

Spatial SM anomalies give a good representation of drought events and their evolution. Below monthly SM anomalies can be plotted for a particular year and country in Europe, from the C3S v202212 data set.

**Try:**

* **Changing the year and study area (e.g., the 2022 summer drought in Italy) to plot soil moisture anomalies for.**
* **Finding patterns in the data, explaining outliers or data gaps.**

In [6]:
mask = xr.open_dataset('./grid_eu.nc')['country'].sortby('lat', ascending=False)
lut = dict(zip(mask.attrs['flag_meanings'], mask.attrs['flag_values']))

def load_study_area(country, ds, var='sm', time_from='2010-01-01'):
    subset = ds.sel(time=slice(time_from, None))[var]
    sel = mask.where(mask==lut[country], drop=True)
    subset = subset.sel(lat=sel.lat, lon=sel.lon, tolerance=.25, method="nearest")
    subset.values[:, ~np.isfinite(sel)] = np.nan
    return subset

country_data = dict()

dropdown = widgets.Dropdown(options=utils.europe_countries, value='Poland',description='Country:')
slider = widgets.SelectionSlider(options=np.arange(2010,2024), value=2022, description='Select a year:', continuous_update=False)
@widgets.interact(year=slider, country=dropdown)
def plot_months(year=2022, country='Poland'):
    global country_data
    
    if not (country in country_data.keys()):
        country_data = {country: (load_study_area(country, c3s_europe, var='sm_anom'))}

    dat = country_data[country].sel(time=slice(f'{year}-01-01', f'{year}-12-31'))
    p = dat.plot(transform=ccrs.PlateCarree(),
                 col='time', col_wrap=4, vmin=-0.15, vmax=0.15, cmap=plt.get_cmap('RdBu'),
                 subplot_kws={'projection': ccrs.PlateCarree()})
    for ax in p.axs.flat:
        ax.axes.add_feature(cartopy.feature.BORDERS, linewidth=0.5, zorder=2)
        ax.axes.add_feature(cartopy.feature.RIVERS, zorder=2)        
        ax.axes.coastlines()

interactive(children=(SelectionSlider(continuous_update=False, description='Select a year:', index=12, options…

## Mapped Quantiles

A simple way to classify potential droughts is by applying thresholds based on quantiles of anomalies in a given time series. Extremely low anomaly values therefore correspond to droughts, while anomalies around and above zero do not. 

Long time series make the analysis more robust, as the sampling becomes denser. The default values are taken from literature.

**Try:**

* **Changing the theshold values for the quantile-based discretization function and find a classification for the time series you have chosen above.**

Which areas have been most affected by the 2022 summer drought?

*Note: You can always reset the interactive controls and change the country by re-running the cell.*

In [67]:
df = anomaly_ts.copy()

drought_levels = {0: 'SM deficit', 1: 'Mild Drought', 2: 'Moderate Drought', 
                  3: 'Significant Drought', 4: 'Severe Drought', 5: 'Extreme Drought', }

@widgets.interact(
mild=widgets.FloatSlider(value=0.35, min=0, max=0.5, step=0.05, description=' [quantile]', style={'description_width': 'initial'},layout=widgets.Layout(width='30%')),
moderate=widgets.FloatSlider(value=0.3, min=0, max=0.5, step=0.05, description=' [quantile]', style={'description_width': 'initial'},layout=widgets.Layout(width='30%')),
significant=widgets.FloatSlider(value=0.25, min=0, max=0.5, step=0.05, description=' [quantile]', style={'description_width': 'initial'},layout=widgets.Layout(width='30%')),
severe=widgets.FloatSlider(value=0.15, min=0, max=0.5, step=0.05, description=' [quantile]', style={'description_width': 'initial'},layout=widgets.Layout(width='30%')),
extreme=widgets.FloatSlider(value=0.1, min=0, max=0.5, step=0.05, description=' [quantile]', style={'description_width': 'initial'}, layout=widgets.Layout(width='30%')),
month=widgets.SelectionSlider(options=np.arange(1, 13), value=6, description='Select a 2022 month to plot:', continuous_update=False, style={'description_width': 'initial'}),
continuous_update=False)
def plot(mild=0.2, moderate=0.1, significant=0.05, severe=0.02, extreme=0.01, month=6):
    bins = df['anomaly'].quantile([0, extreme, severe, significant, moderate, mild, .5]).values

    fig = plt.figure(figsize=(12, 12))
    gs = GridSpec(2, 1, height_ratios=(1,4), hspace=.3)
    
    ax = fig.add_subplot(gs[0, 0],)
    ax.axhline(0, color='k', linestyle='--')
    ax.axhspan(bins[4], bins[5], color='#fee5d9', alpha=0.5, label='mild')
    ax.axhspan(bins[3], bins[4], color='#fcbba1', alpha=0.5, label='moderate')
    ax.axhspan(bins[2], bins[3], color='#fc9272', alpha=0.5, label='significant')
    ax.axhspan(bins[1], bins[2], color='#fb6a4a', alpha=0.5, label='severe')
    ax.axhspan(bins[0], bins[1], color='#de2d26', alpha=0.5, label='extreme')
    
    p = df['anomaly'].plot(marker='o', color='black', ax=ax, ylabel='SM Anomaly $[m³/m³]$', 
                           title=f"Soil Moisture Anomaly drought classification at Lon: {extracted_ts['lon']}°, Lat: {extracted_ts['lat']}°")
    plt.legend()
    
    ax_map = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree())
    ax_map.axes.add_feature(cartopy.feature.BORDERS, linewidth=0.5, zorder=2)
    ax_map.axes.add_feature(cartopy.feature.RIVERS, zorder=2) 
    ax_map.axes.coastlines()
    
    gl = ax_map.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)

    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    selected_country = list(country_data.keys())[0]
    month_data = country_data[selected_country].sel(time=datetime.datetime(2022, month, 1))
    mapplot = month_data.plot(transform=ccrs.PlateCarree(), levels=np.round(bins, 2), cmap=plt.get_cmap('hot'), 
        subplot_kws={'projection': ccrs.PlateCarree()}, add_colorbar=False)
    
    binned = np.digitize(month_data, bins)
    print("Percentage area of drough levels [%]:"
          "\n=====================================")
    for b in range(len(drought_levels)):
        percent_area = np.sum(binned==(b + 1))/binned.size * 100
        print(f"{drought_levels[b]} {percent_area:.1f}")
    
    cax = fig.add_axes([ax_map.get_position().x0, ax_map.get_position().y0-.05, ax_map.get_position().width, 0.02])
    cbar = fig.colorbar(mapplot, ax=ax, cax=cax, orientation="horizontal")
    cbar.set_label('Selected drought levels (in SM anomalies, m³/m³)')

interactive(children=(FloatSlider(value=0.35, description=' [quantile]', layout=Layout(width='30%'), max=0.5, …

## Z-Scores
Z-scores are a way of standardising values from different normal distributions. Z-scores show the number of standard deviations from the mean of the sample.

$\Large Z = \frac{x - \mu}{\sigma}$

In the case of soil moisture anomalies, the distributions of values within a year can vary, e.g., due to distinct seasonal hydrological regimes, or due to differences in data coverage (i.e. a different distribution for values in summer than for those in winter). This can affect the computed climatology and therefore the derived anomalies. By taking into account the standard deviation of samples used to compute the climatologies, we can normalize the anomalies and reduce intra-annual biases.

$\Large Z_{k,i} = \frac{SM_{k,i} - \overline{SM_i}}{\sigma_{i}}$

The default values in the example are taken from literature.

**Try:**

* **Compare the Z-score time series to the anomaly time series above.**
* **Change the threshold values for the quantile-based discretization function find a meaningful classification for the time series you have chosen above.**
* **Extract a time series in areas where data coverage varies strongly within a year and compare the output.**


In [16]:
df = anomaly_ts.copy()

@widgets.interact(
moderate=widgets.FloatSlider(value=1, min=0, max=10, step=0.25, description='Moderate Drought $[\sigma]$', style={'description_width': 'initial'},),
extreme=widgets.FloatSlider(value=2, min=0, max=10, step=0.25, description='Extreme Drought $[\sigma]$', style={'description_width': 'initial'},),
continuous_update=False)
def plot(moderate, extreme):
    df.loc[:, 'zscore'] = (df['soil_moisture'] - df['climatology']) / df['climatology_std']
    bins = [df['zscore'].min(), -extreme, -moderate, 0]

    fig, ax = plt.subplots(1,1, figsize=(15, 5))
    ax_anom = ax.twinx()
    ax.axhline(0, color='k', linestyle='--')
    ax.axhspan(bins[1], bins[2], color='#fc9272', alpha=0.5, label='moderate')
    ax.axhspan(bins[0], bins[1], color='#de2d26', alpha=0.5, label='extreme')
    df['zscore'].plot(marker='o', color='black', ax=ax, ylabel='z score [-]', 
                      title=f"Soil Moisture Z-Score drought classification (Lon: {extracted_ts['lon']}°, Lat: {extracted_ts['lat']}°)")
    ax_anom.plot(df['anomaly'].index, df['anomaly'], color='blue', linewidth=1, label="anomaly")
    ax_anom.set_ylabel('SM Anomaly [m³/m³]')
    
    ax.legend(loc="upper left")
    ax_anom.legend(loc="upper right")

interactive(children=(FloatSlider(value=1.0, description='Moderate Drought $[\\sigma]$', max=10.0, step=0.25, …

## SMASI

One index that was shown in the lecture is the Standardised Vegetation Optical Depth Index [(Moesinger et al., 2022)](https://doi.org/10.5194/bg-19-5107-2022).

Similarly, a Soil Moisture Anomalies Standardized Index (SMASI) can be obtained with a similar method. Compared to the C3S SM-derived indices shown before, SMASI is more robust to noise variations between merging periods (i.e., variations of the number of sensors in the merging). 

The main innovation is represented by the fact that the empirical multivariate distribution of the indices is scaled to a uniform distribution, which renders the time series more consistent in time. The principle is shown in Figure 2 of Moesinger et al. (2022):

<img src="img/svodi_bivariate_distribution.png" width="90%"/>

In [9]:
smasi_europe = xr.open_dataset('./LTC_DATA/STACK_C3S-SOILMOISTURE_EUROPE_SMASI_5x_DAILY.nc')
display(smasi_europe)

<xarray.Dataset>
Dimensions:         (lon: 240, time: 365, lat: 160)
Coordinates:
  * lon             (lon) float64 -9.875 -9.625 -9.375 ... 49.38 49.62 49.88
  * time            (time) datetime64[ns] 2022-01-01 2022-01-02 ... 2022-12-31
  * lat             (lat) float64 69.88 69.62 69.38 69.12 ... 30.62 30.38 30.12
    reference_time  (time) datetime64[ns] ...
Data variables:
    SMASI           (time, lat, lon) float64 ...

The SMASI data set used here contains a harmonised index from 5 satellites, covering the 2007-2022 period. The main differences (synthetically) are that:

* The standardization is performed on a day-of-year basis, with a rolling 90-days window.
* The multivariate distribution is scaled, as mentioned above

The plot below is showing the percentag area of Poland with a under(over)-saturated SM, corresponding to SMASI values below(over) the selected threshold. For refernce, the median (smoothed) SMASI over the area is given.

**Try:**

* **Changing the magnitude of the index threshold to see the affected area in Poland**
* **Changing the level of smoothing to see if intra/inter-annual patterns emerge**

Which years show the most extreme events? Can you see temporal patterns and are they as you would expect them? 

In [10]:
slider = widgets.FloatSlider(min=.5, max=2.5, value=1.5, step=0.5, style={'description_width': 'initial'}, description='Index threshold [-]:', layout=widgets.Layout(width='30%'), continuous_update=False)
slider2 = widgets.IntSlider(min=1, max=36, value=4, step=2, style={'description_width': 'initial'}, description='Median smoothing (# months):', layout=widgets.Layout(width='30%'), continuous_update=False)
@widgets.interact(thresholds=slider, smoothing=slider2)
def plot(thresholds, smoothing):
    df = pd.read_csv(f'./LTC_DATA/mean_ts/smasi_poland.csv', index_col=0, parse_dates=True)
    df = df.rolling(smoothing * 30, min_periods=0).median()
    fig, ax = plt.subplots(1,1, figsize=(15, 7))
    
    ax.fill_between(df.index, -df[f"SMASI_below_{.5:.1f}"], df[f"SMASI_above_{.5:.1f}"], color="#F9AFBC", label=f"|SMASI| > {.5}")
    ax.fill_between(df.index, -df[f"SMASI_below_{thresholds:.1f}"], df[f"SMASI_above_{thresholds:.1f}"], color="#6495ED", label=f"|SMASI| > {thresholds}")
    ax.set_ylabel("Percentage area (%)")
    ax.set_ylim([-100, 100])
    ax.set_yticks(np.linspace(-100, 100, 11))
    ax.set_yticklabels(np.abs(np.linspace(-100, 100, 11)).astype(int))
    
    ax_mean = ax.twinx()
    df["SMASI"].plot(ax=ax_mean, c="blue", linestyle="--", linewidth=1.5, label="SMASI smoothed median")
    ax_mean.set_ylim([-2, 2])
    
    ax.set_xlim([datetime.datetime(2010,1,1), datetime.datetime(2023,1,1)])
    
    ax.set_title("Under/Over-saturated area in Poland")
    ax.hlines(0, datetime.datetime(2010,1,1), datetime.datetime(2023,1,1), color="r", lw=1)
    ax.legend(loc="lower right")
    ax_mean.legend(loc="upper left")

interactive(children=(FloatSlider(value=1.5, continuous_update=False, description='Index threshold [-]:', layo…

We can now compare SMASI to the SM Anomalies over a selected country area to see what is their agreement. The values are averaged in the Latitude dimension to ease the visualization.

Bear in mind that SMASI is provided in daily resolution, whereas the C3S SM data used here is at monthly intervals.

**Try:**

* **Changing the area below to understand whether SMASI and regular SM anomalies agree with each other.**

Where do the differences between the two originate?

In [17]:
dropdown = widgets.Dropdown(options=utils.europe_countries, value='Poland',description='Country:')
slider = widgets.SelectionSlider(options=np.arange(2010,2024), value=2022, description='Select a year:', continuous_update=False)
@widgets.interact(year=slider, country=dropdown)
def plot_hovmoeller(country='Poland'):
    global country_data
    
    if not (country in country_data.keys()):
        country_data = {country: (load_study_area(country, c3s_europe, var='sm_anom'))}
    
    dat = country_data[country]
    dat_2022 = dat.sel(time=slice(datetime.datetime(2021,12,31), datetime.datetime(2023,1,1)))
    smasi_dat = load_study_area(country, smasi_europe, var='SMASI')
    dat_2022 = dat_2022.reindex_like(smasi_dat, method="ffill")  # match the time of the SMASI and SM Anom data sets
    
    fig = plt.figure(figsize=(12, 10))
    gs = GridSpec(3, 1, height_ratios=(1,3,3), hspace=.2)
    
    ax = fig.add_subplot(gs[0, 0],)
    ax.set_xticks([])
    ax_hov_smasi = fig.add_subplot(gs[1, 0],)
    ax_hov_smasi.set_xticks([])    
    ax_hov_anom = fig.add_subplot(gs[2, 0],)
    
    mean = dat_2022.mean(("lon", "lat"))
    std = dat_2022.std(("lon", "lat"))
    
    mean_smasi = smasi_dat.mean(("lon", "lat"))
    std_smasi = smasi_dat.std(("lon", "lat"))
    
    ax.plot(mean.time, mean, label="SM Anomalies")
    ax.fill_between(mean.time, mean - std, mean + std, alpha=.4)
    
    ax_anom = ax.twinx()
    ax_anom.plot(mean_smasi.time, mean_smasi, label="SMASI", color="orange")
    ax_anom.fill_between(mean_smasi.time, mean_smasi - std_smasi, mean_smasi + std_smasi, alpha=.4, color="orange")
    ax.legend(loc="upper left")
    ax_anom.legend(loc="upper right")
    
    ax.set_ylabel("SM Anomalies [m³/m³]")
    ax_anom.set_ylabel('SMASI [-]')
    
    smasi_plot = smasi_dat.mean("lon").transpose("lat", "time").plot(
        ax=ax_hov_smasi, add_colorbar=False, cmap="RdBu")
    ax_hov_smasi.set_xlabel('')
    ax_hov_smasi.set_ylabel('Latitude (deg.)')
    
    anom_plot = dat_2022.mean("lon").transpose("lat", "time").plot(
        ax=ax_hov_anom, add_colorbar=False, cmap="RdBu")
    ax_hov_anom.set_ylabel('Latitude (deg.)')
    
    ax_hov_anom.set_title("SM Anomalies 2022 evolution")
    ax_hov_smasi.set_title("SMASI 2022 evolution")
    
    cax = fig.add_axes([ax_hov_anom.get_position().x0, ax_hov_anom.get_position().y0-.1, ax_hov_anom.get_position().width/3, 0.02])
    cbar = fig.colorbar(smasi_plot, ax=ax, cax=cax, orientation="horizontal")
    cbar.set_label('SMASI [-]')
    
    cax_anom = fig.add_axes([ax_hov_anom.get_position().x0 + ax_hov_anom.get_position().width/3*2, ax_hov_anom.get_position().y0-.1, ax_hov_anom.get_position().width/3, 0.02])
    cbar_anom = fig.colorbar(anom_plot, ax=ax, cax=cax_anom, orientation="horizontal")
    cbar_anom.set_label('SM Anomalies [m³/m³]')

interactive(children=(Dropdown(description='Country:', index=27, options=('Albania', 'Austria', 'Belarus', 'Be…

## Standardized Precipitation Index

Here we compute SPI values for the previously extracted precipitation time series (see first cell in this chapter).
SPI can be computed for different integration time lengths.

SPI values can be aggregated to get an estimate for the duration, severity and intensity of a drought.

1) **Duration**: Number of consecutive time stamps where SPI < 0
2) **Severity**: $\sum{SPI}$ over the duration of a drought
3) **Intensity**: $ \frac{Severity}{Duration}$

**Try:**
* **Changing the integration time length and see how it affects the SPI time series**
* **Choosing a different location in the data extraction cell above, re-run this cell and see how SPI changes**

In [12]:
from scipy.stats import gamma, norm

ts = extracted_ts['data'].copy().to_xarray()

def calc_zscore(ts):
    fit_alpha, fit_loc, fit_beta = gamma.fit(ts)
    pvals = gamma(fit_alpha, loc=fit_loc, scale=fit_beta).cdf(ts)
    zscore = norm.ppf(pvals)
    return zscore

def calc_spi_integrationtime(ts, integrationtime=None):
    if (integrationtime is not None) and (integrationtime!=0):
        ts = ts.rolling(time=integrationtime).mean().dropna("time")
    spi = ts.groupby('time.month').map(calc_zscore)
    return spi

def index_metrics(index):
    duration = [0]
    severity = [0]

    for i in index:
        if i < 0:
            duration.append(duration[-1] + 1)
            severity.append(severity[-1] + i)
        else:
            duration.append(0)
            severity.append(0)
    
    duration = np.array(duration[1:])
    severity = np.array(severity[1:])
    
    return duration, severity


@widgets.interact(integration_time=widgets.Dropdown(options=[0, 1, 2, 3, 12, 48], value=12, description='Integration Time [Months]:', style={'description_width': 'initial'}))
def plot_spi(integration_time):
    spi = pd.DataFrame(data={'index': calc_spi_integrationtime(ts['precipitation'], integration_time).to_pandas()})
    
    duration, severity = index_metrics(spi['index'])
    
    spi.loc[:, 'duration'] = duration
    spi.loc[:, 'severity'] = severity
    spi.loc[:, 'intensity'] = spi['severity'] / spi['duration']
    
    fig, axs = plt.subplots(2, 2, figsize=(12, 5))
    
    axs[0, 0].fill_between(spi['index'].index, spi['index'].values, where=spi['index'].values>=0, color='blue')
    axs[0, 0].fill_between(spi['index'].index, spi['index'].values, where=spi['index'].values<0, color='red')
    axs[0, 0].set_ylabel('SPI[-]')
    axs[0, 0].set_title(f"SPI-{integration_time} at Lon: {extracted_ts['lon']}°, Lat: {extracted_ts['lat']}°")
    
    spi['intensity'].fillna(0).plot(title=f'Intensity', ax=axs[1, 0], ylabel='Severity / Duration')
    spi['duration'].plot(title=f'Duration', ax=axs[0, 1], ylabel='months')
    spi['severity'].plot(title=f'Severity', ax=axs[1, 1], ylabel="$\sum{SPI}$")
    plt.tight_layout()


interactive(children=(Dropdown(description='Integration Time [Months]:', index=4, options=(0, 1, 2, 3, 12, 48)…

# SMASI and SPI comparison over Poland

The various drought indicators (Duration, Intensity, Severity) can be computed using a standardized SM anomaly index like SMASI, the same way it is done for precipitation. The relationship of the two can be explored by considering the averaged monthly SMASI and SPI values over Poland for the 2011-2022 period.

**Try:**
* **Looking at the various drought indicators for the two indices**

Are the two indices comparable? What can be observed on the precipitation/soil moisture deficit relationship?

In [13]:
@widgets.interact(indicator=widgets.Dropdown(options=["Duration", "Intensity", "Severity"], value='Duration',description='Drought indicator:'))
def plot_spi_smasi_comparison(indicator):
    tp_poland = pd.read_csv(f'./LTC_DATA/mean_ts/tp_poland.csv', index_col=0, parse_dates=True)["tp"].to_xarray()
    spi_df = pd.DataFrame(data={'index': calc_spi_integrationtime(tp_poland, 1).to_pandas()})  
    
    smasi_df = pd.read_csv(f'./LTC_DATA/mean_ts/smasi_poland.csv', index_col=0, parse_dates=True).resample("1M").mean()
    
    duration, severity = index_metrics(spi_df["index"])
    
    spi_df.loc[:, 'duration'] = duration
    spi_df.loc[:, 'severity'] = severity
    spi_df.loc[:, 'intensity'] = spi_df['severity'] / spi_df['duration']
    
    duration, severity = index_metrics(smasi_df["SMASI"])
    
    smasi_df.loc[:, 'duration'] = duration
    smasi_df.loc[:, 'severity'] = severity
    smasi_df.loc[:, 'intensity'] = smasi_df['severity'] / smasi_df['duration']
    
    units_lut = dict(duration="Months", intensity="$\sum{SPI}$", severity="Severity / Duration")
    
    fig, axs = plt.subplots(1, 1, figsize=(12, 4))    
    spi_df[indicator.lower()].fillna(0).plot(title=indicator, ax=axs, ylabel=units_lut[indicator.lower()], label="SPI")
    smasi_df[indicator.lower()].fillna(0).plot(title=indicator, ax=axs, label="SMASI")
    
    axs.set_xlim([datetime.datetime(2010,1,1), datetime.datetime(2023,1,1)])
    
    plt.legend()
    plt.tight_layout()

interactive(children=(Dropdown(description='Drought indicator:', options=('Duration', 'Intensity', 'Severity')…

# Soil Moisture / VOD Lag analysis

For this final example use pre-processed, daily, average VOD and soil moisture time series for larger study areas (bounding boxes around Poland, France, Italy, Spain and Europe). We then compute cross-correlation values between the soil moisture and VOD time series with a number of lags applied to the soil moisture data. 

<img src="img/study_areas.png" width="50%"/>

**Try:**
* **Changing the study area and see if you find differences in the computed cross correlations**.
* **Changing the range of lags that is applied to the soil moisture data (positive values refer to shifts forward in time)**
* **What causes these patterns and when do they repeat?**

In [79]:
def crosscorr(vod, sm, lag=0):
    return vod.corr(sm.shift(lag))

slider = widgets.IntRangeSlider(min=-366, max=366, value=[0, 200], step=1, style={'description_width': 'initial'}, description='Lags [days] (min, max):', layout=widgets.Layout(width='30%'), continuous_update=False)
dropdown = widgets.Dropdown(options=['Europe', 'France', 'Spain', 'Italy', 'Poland'], value='Poland', description='Study area:', style={'description_width': 'initial'})
@widgets.interact(study_area=dropdown,lags=slider)
def plot(study_area, lags, Anomalies: bool = False):
    vod_var, sm_var = 'anomaly' if Anomalies else 'vod_K', 'sm_anomaly' if Anomalies else 'sm'
    df_vod = pd.read_csv(f'./LTC_DATA/mean_ts/vod_{study_area.lower()}.csv', index_col=0, parse_dates=True)[vod_var]
    df_sm = pd.read_csv(f'./LTC_DATA/mean_ts/c3s_{study_area.lower()}.csv', index_col=0, parse_dates=True)[sm_var]
    df = pd.concat([df_vod.loc['1988-01-01':'2020-12-31'], df_sm['1988-01-01':'2020-12-31']], axis=1)
    x = np.arange(lags[0], lags[1], 1)
    plt.plot(x, np.array([crosscorr(df[vod_var], df[sm_var], lag=lag) for lag in x]))
    plt.title(f"SM / VOD {'anomaly' if Anomalies else ''} cross-correlation ({study_area})")
    plt.xlabel('Lag [days]')
    plt.ylabel('R[-]')

interactive(children=(Dropdown(description='Study area:', index=4, options=('Europe', 'France', 'Spain', 'Ital…